# Acquisition and Preparation of Zillow Data

### Import libraries and modules

In [1]:
from acquire import get_zillow_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

### Load dataset

#### First, I explored and then queried the dataset in MySQL.  I had to find out how to pull the requested columns from the different tables within the dataset.  The key was knowing that propertylandusetypeid = 261 for single family homes.

In [ ]:
df = get_zillow_data()
df.info(show_counts = True)

In [ ]:
df.isnull().sum()

There are a number of null values throughout the dataset.  Let's see how many records removing all nulls has on the dataset, then remove the biggest one, yearbuilt, and see what that does to the overall set.

In [ ]:
df.dropna().shape

In [ ]:
df = df[df['yearbuilt'].notna()]
df.isnull().sum()

I will keep working my way down.  The only concern I have here is am losing quality rows where just a small piece of info is missing that could easily be imputed.

In [ ]:
df = df[df['taxamount'].notna()]
df.isnull().sum()

In [ ]:
df = df[df['calculatedfinishedsquarefeet'].notna()]
df.isnull().sum()

In [ ]:
df = df[df['taxvaluedollarcnt'].notna()]
df.isnull().sum()

In [ ]:
df.shape

#### The dropna on the enture dataset worked exactly like removing nans from each category individually.

What percentage of the entire dataset is null, and can we simply drop them and move on?

In [ ]:
print(f' Percent loss if all null columns eliminated = {100*(1-(2140235/2152863)):.2f}%')

That is such a small number let's just drop all of these columns.

#### Converting floats to integers, where able

I noticed all of the columns, except bathroomcnt, are floats that could be integers.  So let's change them to integers since that is easier for the computer to work with.

In [ ]:
df = df.astype({'bedroomcnt':'int', 'calculatedfinishedsquarefeet':'int', 'taxvaluedollarcnt':'int', 'yearbuilt':'int', 'taxamount':'int','fips':'int'})
df.info()

#### Examing values for funky numbers

In [ ]:
df.describe().T

In [ ]:
df[df['calculatedfinishedsquarefeet'] < 400]

I am going to eliminate all homes with a square footage under 400 sqft, as it is only 2k rows (< .1% of total) and probably gets rid of some bad data.

In [ ]:
df = df[df['calculatedfinishedsquarefeet'] > 400]

Next I look at the taxvaluedollarcnt.  I decided to eliminate all those under $10k.

In [ ]:
df[df['taxvaluedollarcnt'] < 10000]

In [ ]:
df = df[df['taxvaluedollarcnt'] > 10000]

Lastly, I am going to look at tax amount.

In [ ]:
df[df['taxamount'] < 100]

In [ ]:
for i in range(100,2500,100): 
    x = df[df['taxamount'] < i].shape[0]
    print(f'People who pay less than ${i} tax amount = {x}')

Based on this I am going to cut all those paying less than $200 as outliers.

Now a check on the dataset:

In [ ]:
df = df[df['taxamount'] > 200]
df.describe().T

### So, in summation, the wrangle for the zillow dataset is:
1. Build SQL query for requested features.
2. Acquire SQL query results using an acquire function and local env credentials.  Save to local csv.
3. Drop all nulls
4. Convert all columns to integers, with the exception of bathrooms (half bathrooms force a float)
5. Go through and drop 'funky' data - clear outliers indicating a high probability of being misentered.

## *Test of wrangle*

In [2]:
from wrangle import wrangle_zillow

In [4]:
df = wrangle_zillow()
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2137028.0,3.304882,0.927623,0.0,3.0,3.0,4.0,25.0
bathroomcnt,2137028.0,2.242574,0.988722,0.0,2.0,2.0,3.0,32.0
calculatedfinishedsquarefeet,2137028.0,1865.140382,1221.416753,401.0,1259.0,1624.0,2209.0,952576.0
taxvaluedollarcnt,2137028.0,461122.073688,677382.525670,10091.0,189615.0,328816.0,535000.0,90188462.0
yearbuilt,2137028.0,1960.985765,22.143597,1801.0,1949.0,1958.0,1976.0,2016.0
taxamount,2137028.0,5622.631844,7817.031843,201.0,2547.0,4116.0,6416.0,1078101.0
fips,2137028.0,6048.312721,20.341286,6037.0,6037.0,6037.0,6059.0,6111.0
